In [2]:
import pandas as pd
df = pd.read_csv('/Users/henrysun_1/Desktop/Duke/2024-2025/Summer 2024/fishics/models/Q2A/allseasons.csv')
df = df.drop(['Unnamed: 0'], axis=1)
df

,station_id,project,date,sampling_bout,gear,sample_grp,station,location,replicate,primers,...,White_catfish,Atl_salmon,Longitude,Latitude,Surface_Temp,Bottom_Temp,Surface_Salinity,Bottom_Salinity,temp_strat,salt_strat
0,NJOT_Feb_24_10,NJOT,2/1/24,Feb_24,edna,Trawl,10,B,1,bony,...,0.0,0.0,-74.568000,38.874000,5.956091,5.913677,30.449176,30.469612,0.042414,0.020436
1,NJOT_Feb_24_11,NJOT,2/1/24,Feb_24,edna,Trawl,11,B,1,bony,...,0.0,0.0,-74.618000,38.724000,6.265040,6.349604,30.647220,30.878290,0.084564,0.231070
2,NJOT_Feb_24_12,NJOT,2/1/24,Feb_24,edna,Trawl,12,B,1,bony,...,0.0,0.0,-74.672000,38.728000,6.141931,6.000855,30.286460,30.501550,0.141076,0.215090
3,NJOT_Feb_24_13,NJOT,2/1/24,Feb_24,edna,Trawl,13,B,1,bony,...,0.0,0.0,-74.928000,38.905000,5.563858,5.591211,30.637451,30.595319,0.027353,0.042131
4,NJOT_Feb_24_14,NJOT,2/1/24,Feb_24,edna,Trawl,14,B,1,bony,...,0.0,0.0,-74.969000,38.843000,5.451434,5.624359,29.023967,30.008847,0.172925,0.984880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,NJOT_Aug_23_93,NJOT,8/1/23,Aug_23,edna,Trawl,93,B,1,bony,...,0.0,0.0,-74.828700,38.954683,21.207851,20.596615,31.790128,31.808032,0.611236,0.017904
80,NJOT_Aug_23_94,NJOT,8/1/23,Aug_23,edna,Trawl,94,B,1,bony,...,0.0,0.0,-74.676950,39.141033,20.492487,18.477310,31.893909,31.978538,2.015177,0.084629
81,NJOT_Aug_23_95,NJOT,8/1/23,Aug_23,edna,Trawl,95,B,1,bony,...,0.0,0.0,-74.497300,39.165833,20.549352,16.286042,31.850125,32.115440,4.263310,0.265314
82,NJOT_Aug_23_97,NJOT,8/1/23,Aug_23,edna,Trawl,97,B,1,bony,...,0.0,0.0,-74.023817,39.357500,22.971803,14.291715,31.494135,32.288327,8.680087,0.794192
